In [57]:
import pandas as pd
import numpy as np
import sql_functions as sf
import Capstone_functions as cp
from IPython.display import clear_output

engine = sf.get_engine()
 
schema = "bgg_data"

In [58]:
sql = f'''SELECT * FROM {schema}.main;
      '''
df_main = sf.get_dataframe(sql)

In [59]:
sql = f'''SELECT * FROM {schema}.statistics;
      '''
df_stats = sf.get_dataframe(sql)

In [60]:
sql = f'''SELECT * FROM {schema}.expansions;
      '''
df_expansions = sf.get_dataframe(sql)

In [74]:
sql = f'''SELECT * FROM {schema}.family_bgg; 
      '''
df_family = sf.get_dataframe(sql)

In [85]:
df_family

,id,family_type,family_value,family_id
0,98402,theme,food / cooking,5841
1,98416,creatures,aliens / extraterrestrials,5679
2,98417,game,anima tactics,16800
3,98417,players,two player only games,61979
4,98417,setting,anima universe,1691
...,...,...,...,...
209017,298195,crowdfunding,kickstarter,8374
209018,298195,game,gloomhaven,45610
209019,298195,mechanism,campaign games,24281
209020,298195,mechanism,legacy,25404


In [91]:
mask = df_family['family_value'] == 'kickstarter'

kickstarter_list = df_family.loc[mask,'id']

In [61]:
df_main_stats = pd.merge(df_main,df_stats,on='id')

In [62]:
df_main_stats

,id,yearpublished,min_players,max_players,playtime,min_playtime,max_playtime,min_age,average,user_rated,num_owned,trading,wanting,wishing,numcomments,numweights,averageweight
0,98401,2010,2,4,30,30,30,10,7.08333,3,15,0,1,3,2,2,1.5
1,98402,2011,2,5,30,30,30,8,6.34615,26,20,0,7,13,9,5,2.2
2,98406,2011,2,4,30,30,30,10,6.73333,9,16,3,0,0,3,1,1.0
3,98416,2007,2,4,50,50,50,8,6.80000,2,16,2,1,0,2,1,3.0
4,98417,2009,2,2,60,60,60,12,0.00000,0,10,1,2,2,0,1,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137142,298189,2020,3,7,120,30,120,7,0.00000,0,0,0,0,0,0,0,0.0
137143,298190,2020,3,6,30,30,30,12,6.66667,3,47,2,1,5,4,0,0.0
137144,298191,2022,2,5,80,30,80,8,0.00000,0,5,0,0,0,0,0,0.0
137145,298193,2020,2,6,20,10,20,10,7.38077,26,61,1,0,7,7,2,1.5


In [63]:
mask = (df_main_stats['yearpublished'] >= 2022) & (df_main_stats['num_owned'] <100)

df_new_games = df_main_stats.loc[mask,:]

In [64]:
df_new_games = df_new_games.nlargest(50,"wishing")

In [65]:
expansions_list = df_expansions['expansion_id']

In [92]:
df_new_games['expansion'] = df_new_games['id'].isin(expansions_list)

df_new_games['on_kickstarter'] = df_new_games['id'].isin(kickstarter_list)

In [67]:
columns_to_set_nans = ['yearpublished','min_players','max_players','playtime','min_playtime','max_playtime','averageweight']

for column in columns_to_set_nans:
    mask = df_new_games[column] == 0

    df_new_games.loc[mask,column] = np.nan




In [68]:
df_new_games.drop(['playtime','yearpublished'],inplace=True,axis=1)

In [93]:
df_new_games

,id,min_players,max_players,min_playtime,max_playtime,min_age,average,user_rated,num_owned,trading,wanting,wishing,numcomments,numweights,averageweight,expansion,on_kickstarter
51289,299490,2.0,4.0,150.0,150.0,13,9.04000,25,56,0,193,1375,18,0,NaN,True,False
27445,359871,3.0,4.0,60.0,90.0,14,7.21321,107,94,1,100,1336,109,10,2.7000,True,True
129887,360692,1.0,4.0,50.0,100.0,12,7.93627,51,84,1,82,1132,78,11,3.3636,False,True
78158,354934,1.0,1.0,60.0,60.0,12,8.04545,11,1,0,102,1106,16,4,2.5000,False,True
95854,359438,1.0,4.0,75.0,150.0,12,8.28000,14,13,0,142,1065,28,6,3.6667,False,False
24493,329500,1.0,4.0,45.0,90.0,12,9.04857,7,5,0,85,1060,14,0,NaN,False,True
84742,362944,2.0,4.0,90.0,120.0,12,8.83333,12,10,0,116,1050,11,1,3.0000,False,False
109955,275284,1.0,5.0,60.0,60.0,10,7.74231,52,84,0,76,882,89,2,1.5000,True,True
24950,354570,2.0,2.0,45.0,60.0,14,8.12143,14,11,0,116,870,13,3,1.6667,False,False
117384,298231,2.0,4.0,30.0,90.0,14,7.95472,24,55,1,85,844,54,6,1.6667,False,True
